In [44]:
from dataclasses import fields
from pathlib import Path

import numpy as np
import pyvista as pv

from cardiac_electrophysiology import mesh_processing as mp

In [48]:
patient_id = "01"
input_mesh_file = Path(f"../data/raw/patient_{patient_id}_mesh_with_uacs_fibers_tags.vtk")
output_mesh_file = Path(f"../data/processed/patient_{patient_id}/mesh_with_fibers_tags.vtk")
decimation_factor = 0.75
num_smoothing_iterations = 100

legacy_anatomical_tags = mp.FeatureTags(MV=11, LAA=13, LIPV=21, LSPV=23, RIPV=25, RSPV=27)
new_anatomical_tags = mp.FeatureTags(MV=0, LAA=1, LIPV=2, LSPV=3, RIPV=4, RSPV=5)

In [49]:
input_mesh = pv.read(input_mesh_file)
triangular_mesh = mp.convert_unstructured_to_polydata_mesh(input_mesh)
del triangular_mesh.point_data["alpha"]
del triangular_mesh.point_data["beta"]
for tag_field in fields(legacy_anatomical_tags):
    tag_value = getattr(legacy_anatomical_tags, tag_field.name)
    anatomical_region = np.where(triangular_mesh.cell_data["anatomical_tags"] == tag_value)[0]
    triangular_mesh.cell_data["anatomical_tags"][anatomical_region] = getattr(
        new_anatomical_tags, tag_field.name
    )
triangular_mesh = mp.coarsen_mesh_with_feature_tags(
    triangular_mesh,
    decimation_factor=decimation_factor,
    num_smoothing_iters=num_smoothing_iterations,
)
output_mesh_file.parent.mkdir(parents=True, exist_ok=True)
pv.save_meshio(output_mesh_file, triangular_mesh)

In [50]:
plotter = pv.Plotter(window_size=[900, 900])
plotter.add_mesh(
    triangular_mesh,
    scalars="anatomical_tags",
    cmap="tab20",
    show_edges=True,
    edge_color="lightgray",
    edge_opacity=0.3,
    show_scalar_bar=False,
)
plotter.show()

Widget(value='<iframe src="http://localhost:40667/index.html?ui=P_0x7f7dac812c10_10&reconnect=auto" class="pyv…